In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import json
import os
from request_function import request_url

In [2]:
help(request_url)

Help on function request_url in module request_function:

request_url(*param, req='search', form='json', api_url='http://api.duma.gov.ru/api/', token='26d686e46758658adb812b6924d708e99c8a4706', app_token='app61e9cd3433b223ef90ecdcf059b6ccb48338743c', mode=None)
    Function for API GET-request
    
    Parameters
    ----------
    :param param: 
    List of request parameters objects
        list, tuple
    
    :param req: 
    Type of request
        string, default 'search'
    
    :param form:
    Type of data in response
        string, default 'json'
    
    :param api_url, token, app_token:
    Components of requested url
        default 'http://api.duma.gov.ru/api/', API_KEY, APP_KEY
    
    :param mode:
    If 'print' - is printed text form of response
        string, default None



In [3]:
# список всех депутатов и сенаторов
dep_and_sen_all = request_url('', req='deputies')

200 Success!


In [4]:
dep_and_sen_all.json()

[{'id': '99111772',
  'name': 'Абалаков Александр Николаевич',
  'position': 'Депутат ГД',
  'isCurrent': False,
  'factions': [{'id': '72100004',
    'name': 'Фракция Политической партии "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ"',
    'startDate': '2011-12-04',
    'endDate': '2016-10-04'}]},
 {'id': '99112284',
  'name': 'Абасов Мамед Магарамович',
  'position': 'Депутат ГД',
  'isCurrent': False,
  'factions': [{'id': '72100024',
    'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
    'startDate': '2013-03-06',
    'endDate': '2016-10-04'}]},
 {'id': '99100491',
  'name': 'Абдулатипов Рамазан Гаджимурадович',
  'position': 'Депутат ГД',
  'isCurrent': False,
  'factions': [{'id': '72100020',
    'name': 'Депутатская группа "Российские регионы"',
    'startDate': '1996-01-16',
    'endDate': '1997-11-12'},
   {'id': '72100024',
    'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
    'startDate': '2011-12-04',
    'endDate': '2013-01-27'}

In [5]:
dep_and_sen_all_extracted = pd.io.json.json_normalize(dep_and_sen_all.json())

In [27]:
dep_and_sen_all_extracted

,id,name,position,isCurrent
0,99111772,Абалаков Александр Николаевич,Депутат ГД,False
1,99112284,Абасов Мамед Магарамович,Депутат ГД,False
2,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД,False
3,99100001,Абдулбасиров Магомедтагир Меджидович,Депутат ГД,False
4,99100002,Абельцев Сергей Николаевич,Депутат ГД,False
5,99100635,Абраменков Дмитрий Николаевич,Депутат ГД,False
6,99110156,Абрамов Виктор Семенович,Член СФ,True
7,99111774,Абрамов Иван Николаевич,Член СФ,True
8,99100728,Абрамович Роман Аркадиевич,Депутат ГД,False
9,99111998,Аброськин Николай Павлович,Депутат ГД,False


In [7]:
dep_and_sen_all_extracted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 5 columns):
id           2774 non-null object
name         2774 non-null object
position     2774 non-null object
isCurrent    2774 non-null bool
factions     2103 non-null object
dtypes: bool(1), object(4)
memory usage: 89.5+ KB


## JSON
Результат представляет собой массив объектов, каждый из которых представляет депутата ГД или члена СФ:

**id** — идентификатор депутата ГД или члена СФ

**name** — ФИО

**position** — должность («Депутат ГД» или «Член СФ»)

**isCurrent** — депутат ГД или член СФ в данной должности на данный момент

**factions** — фракция, список, в котором находится словарь либо NaN

In [8]:
factions_with_id = dep_and_sen_all.json()

In [13]:
for i in factions_with_id:
    is_factions = i.get('factions')
    if not is_factions:
        factions_with_id.remove(i)

In [14]:
pd.io.json.json_normalize(factions_with_id).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 5 columns):
id           2102 non-null object
name         2102 non-null object
position     2102 non-null object
isCurrent    2102 non-null bool
factions     2102 non-null object
dtypes: bool(1), object(4)
memory usage: 67.9+ KB


In [15]:
factions_with_id = pd.io.json.json_normalize(factions_with_id, record_path='factions', meta='id',
                                             record_prefix='factions.', errors="ignore")

In [25]:
factions_with_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5031 entries, 0 to 5030
Data columns (total 5 columns):
factions.id           5031 non-null object
factions.name         5031 non-null object
factions.startDate    5031 non-null object
factions.endDate      5031 non-null object
id                    5031 non-null object
dtypes: object(5)
memory usage: 196.6+ KB


In [17]:
dep_and_sen_all_extracted.drop(['factions'], axis='columns', inplace=True)

In [22]:
summary = dep_and_sen_all_extracted.merge(factions_with_id, how='outer')

In [24]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5703 entries, 0 to 5702
Data columns (total 8 columns):
id                    5703 non-null object
name                  5703 non-null object
position              5703 non-null object
isCurrent             5703 non-null bool
factions.id           5031 non-null object
factions.name         5031 non-null object
factions.startDate    5031 non-null object
factions.endDate      5031 non-null object
dtypes: bool(1), object(7)
memory usage: 362.0+ KB


In [28]:
pandas_profiling.ProfileReport(summary)

In [29]:
summary.to_csv(os.path.realpath('input/dep_and_sen_all_extracted_summary.csv'))

## Мапинг всех имен депутатов и айдишников

In [38]:
deputats_mapping = summary[['id', 'name']].copy()

In [39]:
deputats_mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5703 entries, 0 to 5702
Data columns (total 2 columns):
id      5703 non-null object
name    5703 non-null object
dtypes: object(2)
memory usage: 133.7+ KB


In [40]:
deputats_mapping.drop_duplicates(inplace=True)

In [41]:
deputats_mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2774 entries, 0 to 5702
Data columns (total 2 columns):
id      2774 non-null object
name    2774 non-null object
dtypes: object(2)
memory usage: 65.0+ KB


In [42]:
deputats_mapping.to_csv(os.path.realpath('input/members_mapping.csv'))

## Мапинг всех названий фракций и айдишников

In [43]:
fractions_mapping = summary[['factions.id', 'factions.name']].copy()

In [44]:
fractions_mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5703 entries, 0 to 5702
Data columns (total 2 columns):
factions.id      5031 non-null object
factions.name    5031 non-null object
dtypes: object(2)
memory usage: 133.7+ KB


In [45]:
fractions_mapping.drop_duplicates(inplace=True)

In [46]:
fractions_mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 511
Data columns (total 2 columns):
factions.id      27 non-null object
factions.name    27 non-null object
dtypes: object(2)
memory usage: 672.0+ bytes


In [47]:
fractions_mapping

,factions.id,factions.name
0,72100004,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ..."
1,72100024,"Фракция Всероссийской политической партии ""ЕДИ..."
2,72100020,"Депутатская группа ""Российские регионы"""
4,72100001,Фракция Аграрной партии России
5,72100005,Фракция Политической партии ЛДПР - Либерально-...
7,72100011,"Депутаты, не входящие во фракции"
19,72100019,"Депутатская группа ""Народовластие"""
23,72100008,"Фракция ""ЯБЛОКО"""
25,72100022,"Депутатская группа ""Народный депутат"""
31,NaN,NaN


In [48]:
fractions_mapping.to_csv(os.path.realpath('input/fractions_mapping.csv'))